In [1]:
import av
import numpy as np
import tensorflow as tf

ModuleNotFoundError: No module named 'av'

## Using AI to extract features

A pretrained NN model is used to extract features from images.
The "marker" is the center of the bounding box created around the
feature (e.g. around a hand).

In [ ]:
path = '' # path to model
detection_graph = tf.Graph()
sess = {}

# import the model
with detection_graph.as_default():
    od_graph_def = tf.GraphDef()
    with tf.gfile.GFile(path, 'rb') as f:
        serialized = f.read()
        od_graph_def.ParseFromString(serialized)
        tf.import_graph_def(od_graph_def, name='')
    sess = tf.Session(graph=detection_graph)

In [ ]:
path = ''  # path to video file

container = v.open(path)

frames = []
# read the video file
for packet in container.demux():
    for frame in packet.decode():
        if frame.type == 'video':
            frame = frame.to_image()
            frames.append(np.array(frame))

In [ ]:
detected_boxes = []  # list to store the detected objects to
detected_scores = []

# actual detection
for frame in frames:
    image_tensor = detection_graph.get_tensor_by_name('image_tensor:0')
    detection_boxes = detection_graph.get_tensor_by_name('detection_boxes:0')
    detection_scores = detection_graph.get_tensor_by_name('detection_scores:0')
    detection_classes = detection_graph.get_tensor_by_name('detection_classes:0')
    num_detections = detection_graph.get_tensor_by_name('num_detections:0')
    
    frame_expanded = np.expand_dims(frame, axis=0)
    (boxes, scores, classes, num) = sess.run([detection_boxes, detection_scores, detection_classes, num_detections], feed_dict={image_tensor: frame_expanded})
    boxes = np.squeeze(boxes)
    scores = np.squeeze(scores)
    detected_boxes.append(boxes)
    detected_scores.append(scores)

In [ ]:
# calculate the center of the detection boxes, we actually only track this point
thresh = 0.8  # threshold for inclusion

image_dimensions = (0, 0)  # height x width

centers = []
for m, boxes in enumerate(detected_boxes):
    box_centers = []  # centers per image
    for n, box in enumerate(boxes):
        if thresh =< detected_scores[m][n]:
            # calculate center:
            y = ((box[2] - box[0]) * image_dimensions[0]) / 2
            x = ((box[3] - box[1]) * image_dimensions[1]) / 2
            box_centers.append((x, y))
    centers.append(box_centers)

In [ ]:
file = ''  # path to file 
np.save(file, np.array(centers))